In [102]:
import xdrlib
import collections
import warnings
import pandas

In [2]:
#Index for the IDs of additional blocks in the energy file.
#Blocks can be added without sacrificing backward and forward
#compatibility of the energy files.

#For backward compatibility, the order of these should not be changed.        


(enxOR,     # Time and ensemble averaged data for orientation restraints
 enxORI,    # Instantaneous data for orientation restraints
 enxORT,    # Order tensor(s) for orientation restraints
 ensDISRE,  # Distance restraint blocks
 enxDHCOLL, # Data about the free energy blocks in this frame
 enxDHHIST, # BAR histogram
 enxDH,     # BAR raw delta H data
 enxNR      # Total number of extra blocks in the current code,
            # note that the enxio code can read files written by
            # future code which contain more blocks.
) = range(8)

# xdr_datatype
# note that there is no data type 'real' because           
# here we deal with the types as they are actually written to disk.
(xdr_datatype_int, xdr_datatype_float, xdr_datatype_double,
 xdr_datatype_int64, xdr_datatype_char, xdr_datatype_string) = range(6)

In [3]:
enxsubblock = collections.namedtuple('enxsubblock',
                                    ['nr', 'type',
                                     'fval', 'dval', 'ival', 'lval', 'cval', 'sval',
                                     'fval_alloc', 'dval_alloc', 'ival_alloc',
                                     'lval_alloc', 'cval_alloc', 'sval_alloc'])

enxblock = collections.namedtuple('enxblock', ['id', 'nsub', 'sub', 'nsub_alloc'])

enxframe = collections.namedtuple('enxframe', ['t', 'step', 'nsteps', 'dt', 'nsum', 'nre',
                                               'e_size', 'e_alloc', 'ener',
                                               'nblock', 'block', 'nblock_alloc'])

In [69]:
def ndo_int(data, n):
    """mimic of gmx_fio_ndo_int in gromacs"""
    return [data.unpack_int() for i in xrange(n)]


def ndo_float(data, n):
    """mimic of gmx_fio_ndo_float in gromacs"""
    return [data.unpack_float() for i in xrange(n)]


def ndo_double(data, n):
    """mimic of gmx_fio_ndo_double in gromacs"""
    return [data.unpack_double() for i in xrange(n)]


def ndo_int64(data, n):
    """mimic of gmx_fio_ndo_int64 in gromacs"""
    return [data.unpack_huge() for i in xrange(n)]


def ndo_char(data, n):
    """mimic of gmx_fio_ndo_char in gromacs"""
    return [data.unpack_char() for i in xrange(n)]


def ndo_string(data, n):
    """mimic of gmx_fio_ndo_string in gromacs"""
    return [data.unpack_string() for i in xrange(n)]

In [65]:
class Energy(object):
    __slot__ = ['e', 'eav', 'esum']
    
    def __init__(self, e=0, eav=0, esum=0):
        self.e = 0
        self.eav = 0
        self.esum = 0
    
    def __repr__(self):
        return '<{} e={}, eav={}, esum={}>'.format(type(self).__name__,
                                                   self.e, self.eav,
                                                   self.esum)

class SubBlock(object):
    def __init__(self):
        self.nr = 0
        self.type = xdr_datatype_float  # should be double
                                        # if compile in double
        self.val = []
        self.val_alloc = 0
    
    def alloc(self):
        self.val = [0 for _ in range(self.nr)]
        self.vac_alloc = self.nr

        
class Block(object):
    def __init__(self):
        # See enxblock_init
        self.id = enxOR
        self.nsub = 0
        self.sub = []
        self.nsub_alloc = 0


class Frame(object):
    def __init__(self):
        # See init_enxframe
        self.e_alloc = 0
        self.ener = []
        self.nblock = 0
        self.nblock_alloc = 0
        self.block = []
    
    def add_blocks(self, final_number):
        # See add_blocks_enxframe
        self.nblock = final_number
        if final_number > self.nblock_alloc:
            for _ in range(self.nblock_alloc - final_number):
                self.block.append(Block())
            self.nblock_alloc = final_number

In [112]:
#Energy = collections.namedtuple('Energy', 'e eav esum')
#EnerOld = collections.namedtuple('EnerOld', ['bOldFileOpen', 'bReadFirstStep', 'ener_prev'])
EnxNms = collections.namedtuple('EnxNms', 'file_version nre nms bOldFileOpen')
Enxnm = collections.namedtuple('Enxnm', 'name unit')
ENX_VERSION = 5


def edr_strings(data, file_version, n):
    nms = []
    for i in range(n):
        name = data.unpack_string()
        if file_version >= 2:
            unit = data.unpack_string()
        else:
            unit = 'kJ/mol'
        nms.append(Enxnm(name=name, unit=unit))
    return nms


def do_enxnms(data):
    magic = data.unpack_int()
    
    if magic > 0:
        # Assume this is an old edr format
        file_version = 1
        nre = magic
        bOldFileOpen = True
        bReadFirstStep = False
    else:
        bOldFileOpen = False
        if magic != -55555:
            raise ValueError("Energy names magic number mismatch, this is not a GROMACS edr file")
        file_version = ENX_VERSION
        file_version = data.unpack_int()
        if (file_version > ENX_VERSION):
            raise ValueError('Reading file version {} with version {} implementation'.format(file_version, ENX_VERSION))
        nre = data.unpack_int()
    if file_version != ENX_VERSION:
        warnings.warn('Note: enx file_version {}, implementation version {}'.format(file_version, ENX_VERSION))
    nms = edr_strings(data, file_version, nre)
    
    return EnxNms(file_version=file_version, nre=nre, nms=nms, bOldFileOpen=bOldFileOpen)


def do_eheader(data, file_version, fr, nre_test):
    magic = -7777777
    zero = 0
    dum = 0
    tempfix_nr = 0
    ndisre = 0
    startb = 0
    
    bWrongPrecision = False
    bOK = True
    
    first_real_to_check = data.unpack_float()  # should be unpack_real
    if first_real_to_check > -1e-10:
        # Assume we are reading an old format
        file_version = 1
        fr.t = first_real_to_check
        fr.step = data.unpack_int()
    else:
        magic = data.unpack_int()
        if magic != -7777777:
            raise ValueError("Energy header magic number mismatch, this is not a GROMACS edr file")
        file_version = data.unpack_int()
        if file_version > ENX_VERSION:
            raise ValueError('Reading file version {} with version {} implementation'.format(file_version, ENX_VERSION))
        fr.t = data.unpack_double()
        fr.step = data.unpack_hyper()
        fr.nsum = data.unpack_int()
        if file_version >= 3:
            fr.nsteps = data.unpack_hyper()
        else:
            fr.nsteps = max(1, fr.nsum)
        if file_version >= 5:
            fr.dt = data.unpack_double()
        else:
            fr.dt = 0
    fr.nre = data.unpack_int()
    if file_version < 4:
        ndisre = data.unpack_int()
    else:
        # now reserved for possible future use
        data.unpack_int()
    fr.nblock = data.unpack_int()
    assert fr.nblock >= 0
    if ndisre != 0:
        if file_version >= 4:
            raise ValueError("Distance restraint blocks in old style in new style file")
        fr.nblock += 1
    # Frames could have nre=0, so we can not rely only on the fr.nre check
    if (nre_test >= 0
        and ((fr.nre > 0 and fr.nre != nre_test)
             or fr.nre < 0 or ndisre < 0 or fr.nblock < 0)):
        bWrongPrecision = True
        return
    #  we now know what these should be, or we've already bailed out because    
    #  of wrong precision
    if file_version == 1 and (fr.t < 0 or fr.t > 1e20 or fr.step < 0):
        raise ValueError("edr file with negative step number or unreasonable time (and without version number).")
    fr.add_blocks(fr.nblock)
    startb = 0
    if ndisre > 0:
        # sub[0] is the instantaneous data, sub[1] is time averaged
        fr.block[0].add_subblocks(2)
        fr.block[0].id = enxDISRE
        fr.block[0].sub[0].nr = ndisre
        fr.block[0].sub[1].nr = ndisre
        fr.block[0].sub[0].type = dtreal
        fr.block[0].sub[1].type = dtreal
        startb += 1
    # read block header info
    for b in range(startb, fr.nblock):
        if file_version < 4:
            # blocks in old version files always have 1 subblock that          
            # consists of reals.
            fr.block[b].add_subblocks(1)
            nrint = data.unpack_int()
            fr.block[b].id = b - startb
            fr.block[b].sub[0].nr = nrint
            fr.block[b].sub[0].typr = dtreal
        else:
            fr.block[b].id = data.unpack_int()
            nsub = data.unpack_int()
            fr.block[b].nsub = nsub
            fr.block[b].add_subblocks(nsub)
            for sub in fr.block[b].sub:
                typenr = data.unpack_int()
                sub.nr = data.unpack_int()
                sub.type = typenr
    fr.e_size = data.unpack_int()
    # now reserved for possible future use
    data.unpack_int()
    data.unpack_int()
    
    # here, stuff about old versions

    
def do_enx(data, fr):
    file_version = -1
    framenr = 0
    frametime = 0
    try:
        do_eheader(data, file_version, fr, -1)
    except ValueError:
        print("Last energy frame read {} time {:8.3f}         ".format(framenr - 1, frametime))
        raise RuntimeError()
    #print("\rReading energy frame {:6d} time {:8.3f}".format(framenr, fr.t))
    framenr += 1
    frametime = fr.t
    
    bSane = (fr.nre > 0)
    for block in fr.block:
        bSane |= (block.nsub > 0)
    if not (fr.step >= 0 and bSane):
        raise ValueError('Something went wrong')
    if fr.nre > fr.e_alloc:
        for i in range(fr.nre - fr.e_alloc):
            fr.ener.append(Energy(0, 0, 0))
        fr.e_alloc = fr.nre
    for i in range(fr.nre):
        fr.ener[i].e = data.unpack_float()  # Should be unpack_real
        if file_version == 1 or fr.nsum > 0:
            fr.ener[i].eav = data.unpack_float()  # Should be unpack_real
            fr.ener[i].esum = data.unpack_float() # Should be unpack_real
            if file_version == 1:
                # Old, unused real
                data.unpack_real()
    
    # Old version stuff to add later
    
    # Read the blocks
    ndo_readers = (ndo_int, ndo_float, ndo_double,
                   ndo_int64, ndo_char, ndo_string)
    for block in fr.block:
        for sub in block.sub:
            try:
                sub.val = ndo_readers[sub.type](data, sub.nr)
            except IndexError:
                raise ValueError("Reading unknown block data type: this file is corrupted or from the future")
            
        


In [129]:
infile = open('/Users/jon/BigFul/trans_F216_400POPC/0.0/equil.edr').read()
data = xdrlib.Unpacker(infile)

In [130]:
enxnms = do_enxnms(data)
enxnms

EnxNms(file_version=5, nre=49, nms=[Enxnm(name='Bond', unit='kJ/mol'), Enxnm(name='G96Angle', unit='kJ/mol'), Enxnm(name='LJ (SR)', unit='kJ/mol'), Enxnm(name='Coulomb (SR)', unit='kJ/mol'), Enxnm(name='COM Pull En.', unit='kJ/mol'), Enxnm(name='Potential', unit='kJ/mol'), Enxnm(name='Kinetic En.', unit='kJ/mol'), Enxnm(name='Total Energy', unit='kJ/mol'), Enxnm(name='Temperature', unit='K'), Enxnm(name='Pressure', unit='bar'), Enxnm(name='Box-X', unit='nm'), Enxnm(name='Box-Y', unit='nm'), Enxnm(name='Box-Z', unit='nm'), Enxnm(name='Volume', unit='nm^3'), Enxnm(name='Density', unit='kg/m^3'), Enxnm(name='pV', unit='kJ/mol'), Enxnm(name='Enthalpy', unit='kJ/mol'), Enxnm(name='Vir-XX', unit='kJ/mol'), Enxnm(name='Vir-XY', unit='kJ/mol'), Enxnm(name='Vir-XZ', unit='kJ/mol'), Enxnm(name='Vir-YX', unit='kJ/mol'), Enxnm(name='Vir-YY', unit='kJ/mol'), Enxnm(name='Vir-YZ', unit='kJ/mol'), Enxnm(name='Vir-ZX', unit='kJ/mol'), Enxnm(name='Vir-ZY', unit='kJ/mol'), Enxnm(name='Vir-ZZ', unit='kJ/m

In [131]:
all_energies = []
all_names = ['Time'] + [nm.name for nm in enxnms.nms]
times = []
fr = Frame()
while True:
    try:
        do_enx(data, fr)
        times.append(fr.t)
        all_energies.append([fr.t] + [ener.e for ener in fr.ener])
    except EOFError:
        break
df = pandas.DataFrame(all_energies, columns=all_names, index=times)
df

,Time,Bond,G96Angle,LJ (SR),Coulomb (SR),COM Pull En.,Potential,Kinetic En.,Total Energy,Temperature,...,LJ-SR:F216-W_WF,Coul-SR:F216-POPC,LJ-SR:F216-POPC,Coul-SR:W_WF-W_WF,LJ-SR:W_WF-W_WF,Coul-SR:W_WF-POPC,LJ-SR:W_WF-POPC,Coul-SR:POPC-POPC,LJ-SR:POPC-POPC,T-System
0,0,1444.594604,2464.187744,-361246.68750,-1707.177979,1646.660278,-357398.40625,55732.343750,-301666.06250,300.133026,...,-1438.588623,0,0.000000,0,-255560.750000,0,-20932.417969,-1707.177979,-83314.921875,300.133026
5,5,8744.523438,3899.298340,-331296.43750,-1505.295898,1597.735718,-318560.18750,57657.273438,-260902.90625,310.499268,...,-1751.512939,0,0.000000,0,-229445.203125,0,-24326.105469,-1505.295898,-75773.625000,310.499268
10,10,8461.435547,3822.180908,-335126.71875,-1603.562134,1520.607544,-322926.06250,57908.917969,-265017.15625,311.854431,...,-1818.371704,0,0.000000,0,-230963.343750,0,-26334.177734,-1603.562134,-76010.835938,311.854431
15,15,8746.255859,3794.425781,-337509.00000,-1543.201782,1499.618774,-325011.87500,57962.433594,-267049.43750,312.142639,...,-1770.464355,0,0.000000,0,-231642.640625,0,-26869.197266,-1543.201782,-77226.687500,312.142639
20,20,8669.252930,3842.934570,-338211.00000,-1433.018799,1440.656250,-325691.15625,58590.019531,-267101.12500,315.522339,...,-1753.588989,0,0.000000,0,-231937.140625,0,-27675.175781,-1433.018799,-76845.101562,315.522339
25,25,9006.260742,3794.366455,-339963.68750,-1535.292847,1419.288330,-327279.06250,58516.632812,-268762.43750,315.127136,...,-1799.366211,0,0.000000,0,-232496.859375,0,-28217.390625,-1535.292847,-77450.070312,315.127136
30,30,8826.211914,3836.213135,-339928.28125,-1500.245361,1369.332397,-327396.78125,58150.660156,-269246.12500,313.156281,...,-1776.051514,0,-0.030060,0,-232773.781250,0,-28369.822266,-1500.245361,-77008.609375,313.156281
35,35,8781.973633,3771.653809,-340930.15625,-1518.425537,1333.947876,-328561.00000,58473.734375,-270087.25000,314.896118,...,-1830.996826,0,-0.282171,0,-233380.750000,0,-28843.289062,-1518.425537,-76874.843750,314.896118
40,40,8893.084961,3672.645020,-340924.21875,-1492.998901,1316.955322,-328534.53125,58280.437500,-270254.09375,313.855164,...,-1842.843628,0,-0.180971,0,-232494.515625,0,-29378.992188,-1492.998901,-77207.687500,313.855164
45,45,8722.625977,3730.426270,-341120.84375,-1499.459961,1289.631104,-328877.62500,58019.539062,-270858.09375,312.450165,...,-1836.912964,0,-0.258878,0,-232606.890625,0,-29503.652344,-1499.459961,-77173.132812,312.450165


In [100]:
for nm, ener in zip(enxnms.nms, fr.ener):
    print(nm.name, ener.e)

('Bond', 8992.5546875)
('G96Angle', 3662.340087890625)
('LJ (SR)', -344131.46875)
('Coulomb (SR)', -1358.5521240234375)
('COM Pull En.', 1.6785013675689697)
('Potential', -332833.4375)
('Kinetic En.', 57768.54296875)
('Total Energy', -275064.90625)
('Temperature', 311.0984802246094)
('Pressure', 4.837432861328125)
('Box-X', 11.555496215820312)
('Box-Y', 11.555496215820312)
('Box-Z', 13.21330738067627)
('Volume', 1764.3662109375)
('Density', 1003.5034790039062)
('pV', 106.25255584716797)
('Enthalpy', -274958.65625)
('Vir-XX', 19915.02734375)
('Vir-XY', -1269.0589599609375)
('Vir-XZ', 1548.4144287109375)
('Vir-YX', -1269.0552978515625)
('Vir-YY', 19473.32421875)
('Vir-YZ', -589.7844848632812)
('Vir-ZX', 1548.4173583984375)
('Vir-ZY', -589.7881469726562)
('Vir-ZZ', 17609.20703125)
('Pres-XX', -12.348636627197266)
('Pres-XY', 21.565017700195312)
('Pres-XZ', -26.70314598083496)
('Pres-YX', 21.56494903564453)
('Pres-YY', -4.921168327331543)
('Pres-YZ', 10.207423210144043)
('Pres-ZX', -26.703

In [54]:
enxnms.bOldFileOpen

False

In [40]:
fr.nsteps

1L

In [41]:
fr.e_alloc

49

In [75]:
a = [] * 5

In [79]:
a = (1, 'a', [2, 3])
a[-1].append(4)
a

(1, 'a', [2, 3, 4])